This notebook uses the GitHub GraphQL API to retrieve relevant
information about one or more GitHub orgs.

This is a simplified version of `scripts/mystery_orgs.py` that can be used to gather basic data about GitHub orgs.

I created it as one way to learn more about orgs that
we believe may have been created outside of our process by various
employees across our business units. We gather the first few members
of the org to help identify employees who can provide more details
about the purpose of the org and how it is used.

Your API key should be stored in a file called gh_key in the
same folder as this script.

The orgs investigated can be specified in the cell below as part of org_list.

In [24]:
import requests
import json
import pandas as pd

# Assumes your GH API token is in a file called gh_key in this directory
with open('gh_key', 'r') as kf:
    api_token = kf.readline().rstrip() # remove newline & trailing whitespace
    
org_list = ["Moonkube", "ModernAppsNinja"]

In [25]:
def make_query():
    return """query OrgQuery($org_name: String!) {
             organization(login:$org_name) {
               name
               url
               createdAt
               updatedAt
               membersWithRole(first: 15){
                 nodes{
                   login
                   name
                   email
                   company
                 }
               }
              }
            }"""

In [26]:
def get_org_data(api_token, org_list):
    import requests
    import json
    import csv

    url = 'https://api.github.com/graphql'
    headers = {'Authorization': 'token %s' % api_token}
    
    org_info_df = pd.DataFrame()
    
    all_rows = [['org_name', 'org_url', 'org_createdAt', 'org_updatedAt', 'people(login,name,email,company):repeat']]
    
    for org_name in org_list:

        row = []
        query = make_query()

        variables = {"org_name": org_name}
        r = requests.post(url=url, json={'query': query, 'variables': variables}, headers=headers)
        json_data = json.loads(r.text)
        
        df_temp = pd.DataFrame(json_data['data']['organization'])
        org_info_df = org_info_df.append(df_temp, ignore_index=True)
        
        for key in json_data['data']['organization']:
            if key == 'membersWithRole':
                for nkey in json_data['data']['organization'][key]['nodes']:
                    row.append(nkey['login'])
                    row.append(nkey['name'])
                    row.append(nkey['email'])
                    row.append(nkey['company'])
            else:
                row.append(json_data['data']['organization'][key])
        all_rows.append(row)
    
    return org_info_df, all_rows

org_info_df, all_rows = get_org_data(api_token, org_list)

In [27]:
# The dataframe where the info can be found.
org_info_df

,name,url,createdAt,updatedAt,membersWithRole
0,moonkube,https://github.com/moonkube,2020-06-29T23:19:15Z,2020-06-29T23:19:15Z,[]
1,ModernAppsNinja,https://github.com/ModernAppsNinja,2020-01-26T22:51:20Z,2020-08-05T09:18:34Z,"[{'login': 'yogendra', 'name': 'Yogendra Rampu..."


In [20]:
# Another way to look at this data is as a list of rows that could be written to a csv file.
all_rows

[['org_name',
  'org_url',
  'org_createdAt',
  'org_updatedAt',
  'people(login,name,email,company):repeat'],
 ['moonkube',
  'https://github.com/moonkube',
  '2020-06-29T23:19:15Z',
  '2020-06-29T23:19:15Z'],
 ['ModernAppsNinja',
  'https://github.com/ModernAppsNinja',
  '2020-01-26T22:51:20Z',
  '2020-08-05T09:18:34Z',
  'yogendra',
  'Yogendra Rampuria - Yogi',
  '',
  '@yugabyte-db',
  'sammcgeown',
  'Sam McGeown',
  '',
  '@vmware',
  'pipuz',
  'Gianni',
  '',
  None,
  'MAHDTech',
  'MAHDTech',
  'MAHDTech@saltlabs.tech',
  '@salt-labs ',
  'afewell',
  'Arthur Fewell',
  '',
  'VMware',
  'ansergit',
  'Anser Arif',
  '',
  '@vmware',
  'afewellvmware',
  None,
  '',
  None,
  'guyzsarun',
  'Sarun Nuntaviriyakul',
  '',
  '@vmware',
  'yashitanamdeo',
  'Yashita Namdeo',
  'yashita.namdeo2000@gmail.com',
  'Shri Vaishnav Vidyapeeth Vishwavidyalaya',
  'shashwatbangar',
  'Shashwat Bangar',
  '',
  'Shri Vaishnav Vidyapeeth Vishwavidyalaya',
  'AttentiveAryan',
  'Attentive A